In [ ]:
#!pip install -r requirements.txt

In [ ]:
# Necessary imports with added missing modules
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import pandas as pd
import os
import zipfile
import tempfile
import requests
from tqdm import tqdm
from matplotlib.colors import LogNorm

try:
    import rioxarray
except ImportError:
    print("rioxarray not installed.")

# Define the path to the shapefile for calculating emissions by country
relative_path_parts = ['..', 'ne_10m_admin_0_countries', 'ne_10m_admin_0_countries.shp']
path_join = os.path.join(*relative_path_parts)
PATH_SHAPEFILE = os.path.abspath(path_join)

print("All modules successfully imported!")




In [ ]:
# URL of the ZIP file
url = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/EDGAR/datasets/v80_FT2022_GHG/CO2/TOTALS/TOTALS_emi_nc.zip"

# Local path for the ZIP file
relative_path_parts = ['TOTALS_emi_nc.zip']
path_join = os.path.join(*relative_path_parts)
# absolute path
local_zip_path = os.path.abspath(path_join)

# Check if the ZIP file exists locally
if not os.path.exists(local_zip_path):
    # Download the ZIP file if it does not exist locally
    print("Downloading ZIP file...")
    response = requests.get(url, stream=True)
    with open(local_zip_path, 'wb') as file:
        for chunk in tqdm(response.iter_content(chunk_size=1024)):
            if chunk:
                file.write(chunk)

# Unzip the archive
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    temp_dir = tempfile.TemporaryDirectory()
    zip_ref.extractall(temp_dir.name)

# Read .nc files
nc_files = [os.path.join(temp_dir.name, file) for file in os.listdir(temp_dir.name) if file.endswith('.nc')]

# Load .nc data
ds_all = xr.open_mfdataset(nc_files, combine='nested', concat_dim='time', use_cftime=True)

# Time coordinate
start_year = 1970
end_year = start_year + len(nc_files) - 1
time_coord = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='YS')

# Assign time coordinate to dataset
ds_all = ds_all.assign_coords(time=('time', time_coord))


In [ ]:
# Read shapefile
countries_gdf = gpd.read_file(PATH_SHAPEFILE)

# Convert to rioxarray-object
emissions_rio = ds_all['emissions'].rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
emissions_rio.rio.write_crs("epsg:4326", inplace=True)

print('Aggregate countries')
data = {}  # A dictionary to directly store emissions
no_data_countries = []  # A list to store countries without data

for index, row in countries_gdf.iterrows():
    country_name = row['NAME']
    country_geom = row['geometry']
    try:
        clipped = emissions_rio.rio.clip([country_geom], countries_gdf.crs)
        # Convert data directly to Pandas Series and store in the dictionary
        data[country_name] = clipped.sum(dim=['lat', 'lon']).to_pandas()
    except rioxarray.exceptions.NoDataInBounds:
        no_data_countries.append(country_name)

# Output countries without data
for country in no_data_countries:
    print(f"No data for {country}.")

# Create the DataFrame, where 'data' already contains the correct Pandas Series
df_emissions = pd.DataFrame(data)

# Set the index to years
df_emissions.index = pd.to_datetime(df_emissions.index).year
